In [7]:
from konlpy.tag import Mecab
mecab = Mecab()

In [8]:
mecab.pos("비타500")

[('비타', 'NNP'), ('500', 'SN')]

# 텍스트 전처리 (Text Preprocessing)

*   텍스트를 자연어 처리를 위해 용도에 맞도록 사전에 표준화 하는 작업
*   텍스트 내 정보를 유지하고, 중복을 제거하여 분석 효율성을 높이기 위해 전처리를 수행



### 1) 토큰화 (Tokenizing)
* 텍스트를 자연어 처리를 위해 분리 하는 것을
* 토큰화는 단어별로 분리하는 "단어 토큰화(Word Tokenization)"와 문장별로 분리하는 "문장 토큰화(Sentence Tokenization)"로 구분

(이후 실습에서는 단어 토큰화를 "토큰화"로 통일하여 칭하도록 한다)

### 2) 품사 부착(PoS Tagging)
* 각 토큰에 품사 정보를 추가
* 분석시에 불필요한 품사를 제거하거나 (예. 조사, 접속사 등) 필요한 품사를 필터링 하기 위해 사용

### 3) 개체명 인식 (NER, Named Entity Recognition)
* 각 토큰의 개체 구분(기관, 인물, 지역, 날짜 등) 태그를 부착
* 텍스트가 무엇과 관련되어있는지 구분하기 위해 사용
* 예를 들어, 과일의 apple과 기업의 apple을 구분하는 방법이 개체명 인식임

### 4) 원형 복원 (Stemming & Lemmatization)
* 각 토큰의 원형 복원을 함으로써 토큰을 표준화하여 불필요한 데이터 중복을 방지 (=단어의 수를 줄일수 있어 연산을 효율성을 높임)
* 어간 추출(Stemming) : 품사를 무시하고 규칙에 기반하여 어간을 추출
* 표제어 추출 (Lemmatization) : 품사정보를 유지하여 표제어 추출

### 5) 불용어 처리 (Stopword)
* 자연어 처리를 위해 불필요한 요소를 제거하는 작업
* 불필요한 품사를 제거하는 작업과 불필요한 단어를 제거하는 작업으로 구성
* 불필요한 토큰을 제거함으로써 연산의 효율성을 높임



---



# 1 영문 전처리 실습


NLTK lib (https://www.nltk.org/) 사용

- tokenining -> tagging -> chunking

## 1.1 실습용 영문기사 수집
온라인 기사를 바로 수집하여 실습데이터로 사용

https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url="https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [7]:
article = soup.select("p")
text = article[3].text

## 1.2 영문 토큰화
https://www.nltk.org/api/nltk.tokenize.html

In [12]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /Users/ryleyun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
word_tokens = word_tokenize("Good muffins cost $3.88 in New York")
print(word_tokens)

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York']


In [30]:
# wordpuncttokenizer는 puctution들도 모두 띄어서 나옴
from nltk.tokenize import WordPunctTokenizer
word_punct = WordPunctTokenizer().tokenize(text)
print(word_punct)

['And', 'yes', ',', 'she', 'does', 'mean', 'everybody', "'", 's', 'job', 'from', 'yours', 'to', 'mine', 'and', 'onward', 'to', 'the', 'role', 'of', 'grain', 'farmers', 'in', 'Egypt', ',', 'pastry', 'chefs', 'in', 'Paris', 'and', 'dog', 'walkers', 'in', 'Oregon', 'i', '.', 'e', '.', 'every', 'job', '.', 'We', 'will', 'now', 'be', 'able', 'to', 'help', 'direct', 'all', 'workers', '’', 'actions', 'and', 'behavior', 'with', 'a', 'new', 'degree', 'of', 'intelligence', 'that', 'comes', 'from', 'predictive', 'analytics', ',', 'all', 'stemming', 'from', 'the', 'AI', 'engines', 'we', 'will', 'now', 'increasingly', 'depend', 'upon', '.']


In [31]:
# TreebankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer
word_punct = TreebankWordTokenizer().tokenize(text)
print(word_punct)

['And', 'yes', ',', 'she', 'does', 'mean', 'everybody', "'s", 'job', 'from', 'yours', 'to', 'mine', 'and', 'onward', 'to', 'the', 'role', 'of', 'grain', 'farmers', 'in', 'Egypt', ',', 'pastry', 'chefs', 'in', 'Paris', 'and', 'dog', 'walkers', 'in', 'Oregon', 'i.e.', 'every', 'job.', 'We', 'will', 'now', 'be', 'able', 'to', 'help', 'direct', 'all', 'workers’', 'actions', 'and', 'behavior', 'with', 'a', 'new', 'degree', 'of', 'intelligence', 'that', 'comes', 'from', 'predictive', 'analytics', ',', 'all', 'stemming', 'from', 'the', 'AI', 'engines', 'we', 'will', 'now', 'increasingly', 'depend', 'upon', '.']


##### - spaCy 라는 tokenizer도 있다

## 1.3 영문 품사 부착 (PoS Tagging)
분리한 토큰마다 품사를 부착한다

https://www.nltk.org/api/nltk.tag.html

태크목록 : https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

In [32]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ryleyun/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [33]:
tagged = pos_tag(word_tokens)
print(tagged)

[('Good', 'JJ'), ('muffins', 'NNS'), ('cost', 'VBP'), ('$', '$'), ('3.88', 'CD'), ('in', 'IN'), ('New', 'NNP'), ('York', 'NNP')]


## 1.4 개체명 인식 (NER, Named Entity Recognition)

http://www.nltk.org/api/nltk.chunk.html

In [26]:
nltk.download("words")
nltk.download("maxent_ne_chunker")

[nltk_data] Downloading package words to /Users/ryleyun/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/ryleyun/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [34]:
from nltk import ne_chunk
ne_token = ne_chunk(tagged)
print(ne_token)

(S
  (GPE Good/JJ)
  muffins/NNS
  cost/VBP
  $/$
  3.88/CD
  in/IN
  (GPE New/NNP York/NNP))


## 1.5 원형 복원
각 토큰의 원형을 복원하여 표준화 한다. 

### 1.5.1 어간추출 (Stemming)

* 규칙에 기반 하여 토큰을 표준화
* ning제거, ful 제거 등

https://www.nltk.org/api/nltk.stem.html

규칙상세 : https://tartarus.org/martin/PorterStemmer/def.txt

In [36]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [38]:
ps.stem("running"); ps.stem("believers")

'believ'

### 1.5.2 표제어 추출 (Lemmatization)

* 품사정보를 보존하여 토큰을 표준화

http://www.nltk.org/api/nltk.stem.html?highlight=lemmatizer

In [39]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /Users/ryleyun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()

In [51]:
wl.lemmatize("running"); wl.lemmatize("beautiful")

'beautiful'

## 1.6 불용어 처리 (Stopword)

In [56]:
stop_pos = ['IN', 'CC', 'DT']
from collections import Counter
tagged = pos_tag(word_tokenize(text))
Counter(tagged).most_common(5)

[((',', ','), 3),
 (('from', 'IN'), 3),
 (('to', 'TO'), 3),
 (('and', 'CC'), 3),
 (('in', 'IN'), 3)]

In [61]:
words = []
for tag in tagged:
    if not tag[1] in stop_pos:
        words.append(tag[0])
words[:5]

['yes', ',', 'she', 'does', 'mean']



---



# 2 한글 전처리 실습
영문은 공백으로 토큰화가 가능하지만, 한글의 경우 품사를 고려하여 토큰화 해야한다.

## 2.1 실습용 한글기사 수집
온라인 기사를 바로 수집하여 실습데이터로 사용

http://news.chosun.com/site/data/html_dir/2018/07/10/2018071004121.html

In [88]:
url = "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=110&oid=023&aid=0003386456"
headers = {"user-agent":"Mozilla/5.0"}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text,'html.parser')

In [97]:
article = soup.select_one("#articleBodyContents").text

## 2.2 한글 토큰화 및 형태소 분석

### Mecab

In [136]:
from konlpy.tag import Mecab
m = Mecab()
tagged_m = m.morphs("암특이적바이오마커발굴및바이오마커에대한프로브개발good")
tagged_m

['암',
 '특이',
 '적',
 '바이오',
 '마커',
 '발굴',
 '및',
 '바이오',
 '마커',
 '에',
 '대한',
 '프로브',
 '개발',
 'good']

### Komoran

In [131]:
from konlpy.tag import Komoran
k = Komoran()
tagged_k = k.pos("암특이적바이오마커발굴및바이오마커에대한프로브개발")
tagged_k

[('암특이적바이오마커발굴및바이오마커에대한프로브개발', 'NA')]

### Okt

In [132]:
from konlpy.tag import Okt
o = Okt()
tagged_o = o.pos("암특이적바이오마커발굴및바이오마커에대한프로브개발")
tagged_o

[('암', 'Modifier'),
 ('특이', 'Noun'),
 ('적', 'Suffix'),
 ('바이오', 'Noun'),
 ('마커', 'Adverb'),
 ('발굴', 'Noun'),
 ('및', 'Noun'),
 ('바이오', 'Noun'),
 ('마커', 'Adverb'),
 ('에', 'Josa'),
 ('대', 'Modifier'),
 ('한프로', 'Noun'),
 ('브', 'Noun'),
 ('개발', 'Noun')]

### Kkma

In [133]:
from konlpy.tag import Kkma
kk = Kkma()
tagged_kk = kk.pos("암특이적바이오마커발굴및바이오마커에대한프로브개발")
tagged_kk

[('암', 'NNG'),
 ('특이', 'NNG'),
 ('적', 'XSN'),
 ('바이오', 'NNG'),
 ('마커', 'NNG'),
 ('발굴', 'NNG'),
 ('및', 'MAG'),
 ('바이오', 'NNG'),
 ('마커', 'NNG'),
 ('에', 'JKM'),
 ('대하', 'VV'),
 ('ㄴ', 'ETD'),
 ('프로브', 'NNG'),
 ('개발', 'NNG')]

### Hannanum

In [134]:
from konlpy.tag import Hannanum
h = Hannanum()
tagged_h = h.pos("암특이적바이오마커발굴및바이오마커에대한프로브개발")
tagged_h

[('암특이적바이오마커발굴및바이오마커에대한프로브개발', 'N')]

한글 자연어처리기 비교

https://konlpy.org/ko/latest/morph/

##### - soynlp도 있다!

## 2.3 한글 품사 부착 (PoS Tagging)

PoS Tag 목록

https://docs.google.com/spreadsheets/u/1/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

In [145]:
from konlpy.tag import Komoran
m = Mecab()
pos_m = m.pos("기존 가속기 설계 및 운전에 있어서 엄밀하게 취급 되지 않았던, 공간전하 효과(선형 및 비선형), x-y coupling 및  x-z coupling, 급격한 에너지 변화 등을 고려한 빔 물리 이론 모형을 개발하고, 국제 핵융합 재료조사시설(IFMIF) 개발에 적용한다")
pos_m

[('기존', 'NNG'),
 ('가속기', 'NNG'),
 ('설계', 'NNG'),
 ('및', 'MAJ'),
 ('운전', 'NNG'),
 ('에', 'JKB'),
 ('있', 'VV'),
 ('어서', 'EC'),
 ('엄밀', 'XR'),
 ('하', 'XSA'),
 ('게', 'EC'),
 ('취급', 'NNG'),
 ('되', 'VV'),
 ('지', 'EC'),
 ('않', 'VX'),
 ('았', 'EP'),
 ('던', 'ETM'),
 (',', 'SC'),
 ('공간', 'NNG'),
 ('전하', 'NNG'),
 ('효과', 'NNG'),
 ('(', 'SSO'),
 ('선형', 'NNG'),
 ('및', 'MAJ'),
 ('비', 'XPN'),
 ('선형', 'NNG'),
 (')', 'SSC'),
 (',', 'SC'),
 ('x', 'SL'),
 ('-', 'SY'),
 ('y', 'SL'),
 ('coupling', 'SL'),
 ('및', 'MAJ'),
 ('x', 'SL'),
 ('-', 'SY'),
 ('z', 'SL'),
 ('coupling', 'SL'),
 (',', 'SC'),
 ('급격', 'XR'),
 ('한', 'XSA+ETM'),
 ('에너지', 'NNG'),
 ('변화', 'NNG'),
 ('등', 'NNB'),
 ('을', 'JKO'),
 ('고려', 'NNG'),
 ('한', 'XSV+ETM'),
 ('빔', 'NNG'),
 ('물리', 'NNG'),
 ('이론', 'NNG'),
 ('모형', 'NNG'),
 ('을', 'JKO'),
 ('개발', 'NNG'),
 ('하', 'XSV'),
 ('고', 'EC'),
 (',', 'SC'),
 ('국제', 'NNG'),
 ('핵융합', 'NNG'),
 ('재료', 'NNG'),
 ('조사', 'NNG'),
 ('시설', 'NNG'),
 ('(', 'SSO'),
 ('IFMIF', 'SL'),
 (')', 'SSC'),
 ('개발', 'NNG'),
 ('에', '

## 2.4 불용어(Stopword) 처리
분석에 불필요한 품사를 제거하고, 불필요한 단어(불용어)를 제거한다

In [146]:
stop_pos = ['EC','EP','SC', 'JK', 'JKO']

In [147]:
words = []
for tag in pos_m:
    if tag[1] in stop_pos:
        continue
    words.append(tag[0])

In [148]:
print(len(pos_m), len(words))

67 56


# 2 N-gram

In [1]:
import nltk
from nltk import bigrams, word_tokenize
from nltk.util import ngrams

In [5]:
sentence = "I am a boy"
tokens = word_tokenize(sentence)
tokens

['I', 'am', 'a', 'boy']

In [22]:
bigram = bigrams(tokens)
trigram = ngrams(tokens, 3, pad_right=True)
# pad_right = True 하면 none과 함께 나옴
# left_pad_symbol = "" -> 이것도 왼쪽 오른 쪽에 심볼도 넣을 수 있음

In [20]:
for token in bigram:
    print(token)
    
for token in trigram:
    print(token)

('I', 'am')
('am', 'a')
('a', 'boy')
('I', 'am', 'a')
('am', 'a', 'boy')
('a', 'boy', None)
('boy', None, None)


In [23]:
list(bigram)

[('I', 'am'), ('am', 'a'), ('a', 'boy')]